In [1]:
# Importing Pandas for data manipulation operations.
import pandas as pd
# Importing NumPy for numerical operations.
import numpy as np


In [2]:
# Importing the datasets from CSV files into pandas dataframes.
fisher_df_1 = pd.read_csv("DATASETS\Fisher_slno.1-101.csv",     low_memory = False)
fisher_df_2 = pd.read_csv("DATASETS\Fisher_slno.102-4291.csv",  low_memory = False)
fisher_df_3 = pd.read_csv("DATASETS\Fisher_slno.4292-7217.csv", low_memory = False)

# Importing the fish labels provided by the BAU team for later use.
fish_labels = pd.read_csv("DATASETS/fish_species.csv", low_memory = False)

In [3]:
# List of months in a specific order, starting from April and ending in March.
MONTHS = [
    'January--Magh',
    'February--Falgun',
    'March--Chaitra',
    'April--Boishakh',
    'May--Jeystho',
    'June--Asharh',
    'July--Srabon',
    'August--Bhadro',
    'September--Ashwin',
    'October--Kartik',
    'November--Aghrahan',
    'December--Poush'
]

# Dictionary mapping numeric keys to various fishing sources from the survey.
SOURCE = {
    1:  "Marsh", 2:  "Haor", 3:  "Canal", 4:  "River",
    5:  "Mohona", 6:  "River (Cultivation)", 7:  "Pond",
    8:  "Seasonal Cultivation", 9:  "Fish Farming in Cages",
    10: "Pen Culture (Net)", 11: "Flooded Reservoirs", 99: "Others"
}

# Dictionary mapping numeric keys to reasons for fish loss from the survey.
REASONS = {
    1: "Damage During Harvesting", 2: "Too Long In Nets (Physical Damage)",
    3: "High Temperature, Delay In Taking To Market", 4: "Not Enough Ice or Insulated Containers",
    5: "Inadequacy of Fish Preservation Materials", 6: "Inadequate Cold Storage Facilities",
    7: "Inadequacy of Communication Systems", 8: "Spoilage of Fish During Transportation",
    9: "Loss of Fish From Unloading & Loading", 10: "Result of Medication Used On Fish",
    99: "Other Unrecorded Reason For Loss"
}

# Dictionary mapping string keys to Distribution Channels from the survey.
DISTRIBUTION = {
    'f': 'Local Agent',
    'b': 'Wholesaler (Dealer)',
    'p': 'Wholesaler (Wholesale)',
    'a': 'Wholesaler (Merchant)',
    'c': 'Merchant (Commission Agent)',
    'k': 'Retailer',
    'v': 'Consumer',
    'h': 'Hotel Restaurant',
    'd': 'Depot Owner',
    'ac': 'Account Holder',
    'r': 'Exporter'
}

In [4]:
# Convert the 'Species_Name' column of the 'fish_labels' DataFrame into a Series.
FISH_LABELS = pd.Series(
    fish_labels.Species_Name.values,
    index = fish_labels.Fish_Species_Serial_Number
).to_dict()

#### <b>Overview of Fishing Techniques Used to Harvest Fish - (Question 3)</b>

In [5]:
# Selecting columns that represent the source of fishing from the three DataFrames.
source_of_fishing1 = fisher_df_1.iloc[:, 22:27]
source_of_fishing2 = fisher_df_2.iloc[:, 22:27]
source_of_fishing3 = fisher_df_3.iloc[:, 22:27]

# Concatenating the source columns from all three DataFrames into one single DataFrame.
source_of_fishing = pd.concat([
    source_of_fishing1, source_of_fishing2, source_of_fishing3
])

# Resetting the index of the new DataFrame.
source_of_fishing.reset_index(drop = True, inplace = True)

In [6]:
# Displaying the DataFrame's shape and size.
display(
    source_of_fishing.shape,
    source_of_fishing.head(5)
)

(7217, 5)

,q3_1,q3_2,q3_3,q3_4,q3_5
0,6.0,NaN,NaN,NaN,NaN
1,6.0,NaN,NaN,NaN,NaN
2,6.0,NaN,NaN,NaN,NaN
3,6.0,NaN,NaN,NaN,NaN
4,6.0,NaN,NaN,NaN,NaN


In [7]:
# Headings of the questions related to source.
headings = ['q3_1', 'q3_2', 'q3_3', 'q3_4', 'q3_5']

# Reshape the DataFrame by melting it, with 'Source' as the values.
data_melting = source_of_fishing.melt(
    value_vars = headings, 
    var_name   = 'Question', 
    value_name = 'Source'
)

# Counting the occurrences of each source.
source_count = data_melting['Source'].value_counts().reset_index()

# Rename the columns to 'Source' and 'Count'.
source_count.columns = ['Source', 'Count']

In [8]:
# Displaying the DataFrame's shape and size.
display(
    source_count.shape,
    source_count.head(5)
)

(25, 2)

,Source,Count
0,6.0,3117
1,4.0,2508
2,1.0,547
3,2.0,378
4,99.0,253


In [9]:
# Mapping the 'Source' values to their descriptions using the 'source' dictionary.
source_count['Source Desc'] = source_count['Source'].map(SOURCE).fillna('Others')

# Group by the 'Source Desc' and sum the 'Count'.
grouped_totals = source_count.groupby('Source Desc')['Count'].sum().reset_index()

# Creating final DataFrame for exporting.
SOURCE_OF_FISHING_DF = grouped_totals.sort_values(by = 'Count', ascending = False)

In [10]:
# Displaying the DataFrame's shape and size.
display(
    SOURCE_OF_FISHING_DF.shape,
    SOURCE_OF_FISHING_DF.head(5)
)

(12, 2)

,Source Desc,Count
10,River (Cultivation),3117
9,River,2508
4,Marsh,547
6,Others,454
3,Haor,378


# Q3_SOURCE_OF_FISHING.csv saved

In [13]:
# Saving the DataFrame to a CSV file.
SOURCE_OF_FISHING_DF.to_csv(
    'DATASETS/Cleaned_Data/Q3_SOURCE_OF_FISHING.csv',
    index = False
)

#### <b>Analysing Annual Catch Volumes and Species-Specific Harvest Data - (Question 4)</b>

In [14]:
# Selecting columns that represent the catch from the three DataFrames.
annual_catch_totals1 = fisher_df_1.iloc[:, 41:181]
annual_catch_totals2 = fisher_df_2.iloc[:, 41:181]
annual_catch_totals3 = fisher_df_3.iloc[:, 41:181]

# Concatenating the catch columns from all three DataFrames into one single DataFrame.
annual_catch_totals = pd.concat([
    annual_catch_totals1, annual_catch_totals2, annual_catch_totals3
])

# Resetting the index of the new DataFrame.
annual_catch_totals.reset_index(drop = True, inplace = True)

In [15]:
# Displaying the DataFrame's shape and size.
display(
    annual_catch_totals.shape,
    annual_catch_totals.head(5)
)

(7217, 140)

,q4_1_n,q4_f_1_1,q4_f_1_2,q4_f_1_3,q4_f_1_4,q4_f_1_5,q4_f_1_6,q4_f_1_7,q4_f_1_8,q4_f_1_9,...,q4_f_10_4,q4_f_10_5,q4_f_10_6,q4_f_10_7,q4_f_10_8,q4_f_10_9,q4_f_10_10,q4_f_10_11,q4_f_10_12,q4_f_10_t
0,1,5000.0,5000.0,5000.0,500.0,500.0,500.0,500.0,800.0,800,...,0,0,0,0,0,0,0,0,0,0
1,1,1200.0,1200.0,1200.0,500.0,500.0,500.0,500.0,500.0,500,...,0,0,0,0,0,0,0,0,0,0
2,1,2000.0,2000.0,2000.0,500.0,500.0,500.0,500.0,500.0,500,...,0,0,0,0,0,0,0,0,0,0
3,1,2400.0,2400.0,2400.0,800.0,800.0,800.0,800.0,800.0,800,...,0,0,0,0,0,0,0,0,0,0
4,1,1500.0,0.0,0.0,0.0,300.0,0.0,300.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Looping through the range (1 -> 10) to generate column names for 'q4_n'.
for x in range(1, 11):
    column_name = f'q4_{x}_n'

    # Checking if the column exists in the 'annual_catch_totals' DataFrame.
    if column_name in annual_catch_totals.columns:
        # Mapping the values to fish labels using the FISH_LABELS dictionary.
        annual_catch_totals[column_name] = annual_catch_totals[column_name].map(FISH_LABELS)

# Looping through the columns in the 'annual_catch_totals' DataFrame.
for z in annual_catch_totals.columns:
    # Checking if the column name contains '_n'
    if '_n' in z:
        # Filling NaN values in the column with 'Other Species'.
        annual_catch_totals[z].fillna('Other Species', inplace = True)

C:\Users\bapti\AppData\Local\Temp\ipykernel_31488\362033000.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  annual_catch_totals[z].fillna('Other Species', inplace = True)


In [17]:
# Creating new column names.
new_columns = ['Fish Name'] + MONTHS + ['Year Total']

# Creating an empty DataFrame with the specified columns above.
annual_catch_df = pd.DataFrame(columns = new_columns)

In [18]:
# Iterating through columns that contain '_n' in their names.
for x in annual_catch_totals.filter(like = '_n').columns:

    # Extracting the series number from the column name.
    series_number = x.split('_')[1]
    # Creating a list of column names for each month based on the series number.
    month_column  = [f'q4_f_{series_number}_{z}' for z in range(1, 13)]

    # Iterating through the unique fish species names in the current column.
    for i in annual_catch_totals[x].unique():
        # Data List.
        data = []

        # Summing of the catch data for each month and converting to metric tonnes.
        for col, month_name in zip(month_column, MONTHS):
            month_info = annual_catch_totals[annual_catch_totals[x] == i][col].sum() / 1000
            data.append(month_info)

        # Calculating the total annual catch for the current species of fish.
        year_total = sum(data)

        # Checking if the species already exists in the 'annual_catch_df'.
        if i in annual_catch_df['Fish Name'].values:
            # If it exists, update the existing row.
            index = annual_catch_df[annual_catch_df['Fish Name'] == i].index[0]
            annual_catch_df.loc[index, MONTHS] += data
            annual_catch_df.loc[index, 'Year Total'] += year_total
        else:
            # If it does not exist, add a new row for the species.
            new_row = [i] + data + [year_total]
            annual_catch_df.loc[len(annual_catch_df)] = new_row

# Calculating the total catch for each month across all fish species.
month_totals_df = annual_catch_df.loc[:, 'January--Magh':'December--Poush'].sum()

# Sorting the DataFrame by 'Year Total' in descending order to get the total catch per species.
species_totals_df = annual_catch_df.sort_values(by = 'Year Total', ascending = False)

In [19]:
# Resetting the index of the 'month_totals_df'.
month_totals_df = month_totals_df.reset_index()

# Renaming the name of the columns.
month_totals_df.columns = ['Month', 'Total']

# Rounding the 'Total' column values to 2 decimal places.
month_totals_df['Total'] = month_totals_df['Total'].round(2)

# Creating final DataFrame for exporting.
MONTHLY_CATCH_DF = month_totals_df

In [20]:
# Rounding all numerical columns in the 'species_totals_df' DataFrame to 2 decimal places.
species_totals_df = species_totals_df.iloc[:, 0:].round(2)

# Sorting the DataFrame by 'Year Total' column in descending order.
species_totals_df = species_totals_df.sort_values(by = 'Year Total', ascending = False)

# Removing rows where the 'Fish Name' column is 'Other Species'.
species_totals_df = species_totals_df[species_totals_df['Fish Name'] != 'Other Species']

# Selecting the Top 10 species based on the 'Year Total'.
MONTHLY_FISH_CATCH_DF = species_totals_df.head(10)

In [21]:
# Displaying the DataFrame's shape and size.
display(
    MONTHLY_CATCH_DF.shape,
    MONTHLY_CATCH_DF.head(5)
)

(12, 2)

,Month,Total
0,January--Magh,6669.97
1,February--Falgun,5987.60
2,March--Chaitra,5083.48
3,April--Boishakh,5326.34
4,May--Jeystho,6931.92


In [22]:
# Displaying the DataFrame's shape and size.
display(
    MONTHLY_FISH_CATCH_DF.shape,
    MONTHLY_FISH_CATCH_DF
)

(10, 14)

,Fish Name,January--Magh,February--Falgun,March--Chaitra,April--Boishakh,May--Jeystho,June--Asharh,July--Srabon,August--Bhadro,September--Ashwin,October--Kartik,November--Aghrahan,December--Poush,Year Total
21,Tilapia,1044.09,883.52,753.40,1142.83,1735.52,1951.20,2245.23,2084.63,2278.03,2133.77,1945.46,1505.43,19703.12
0,Rui,1000.27,908.00,498.74,473.14,693.52,799.03,1204.51,1595.43,1381.04,1442.97,1199.58,1073.06,12269.28
20,Pangas,1057.78,821.35,673.71,350.41,496.35,630.40,1016.08,1182.29,1590.36,1574.62,1664.18,891.76,11949.28
29,Mrigel,457.51,419.13,272.06,287.16,383.16,442.82,587.62,1018.55,948.93,683.34,617.40,607.35,6725.04
13,Silver carp,536.99,446.38,322.56,289.59,448.76,514.94,574.09,772.81,613.78,691.95,753.03,630.02,6594.89
1,Katla,413.16,332.74,211.00,207.02,277.20,369.55,677.43,863.64,838.45,705.36,544.66,426.55,5866.78
22,Hilsha,209.29,214.18,496.20,671.26,696.22,437.88,339.22,313.74,289.05,291.17,135.06,95.35,4188.62
25,Bata,114.36,107.62,89.31,91.02,63.60,136.40,344.64,289.00,291.15,245.24,205.46,185.31,2163.10
32,Koi,95.21,227.32,183.35,126.09,186.83,101.08,85.33,125.75,100.87,87.36,157.59,72.42,1549.21
35,Big-head carp,125.63,107.50,77.18,82.89,106.07,119.26,132.45,123.53,120.72,117.47,156.69,201.06,1470.45


# Q4_MONTHLY_CATCH.csv saved

In [23]:
# Saving the DataFrame to a CSV file.
MONTHLY_CATCH_DF.to_csv(
    'DATASETS/Cleaned_Data/Q4_MONTHLY_CATCH.csv',
    index = False
)

In [24]:
# Saving the DataFrame to a CSV file.
MONTHLY_FISH_CATCH_DF.to_csv(
    'DATASETS/Cleaned_Data/Q4_MONTHLY_FISH_CATCH.csv',
    index = False
)

#### <b>Assessing Yearly Catch Totals from Designated Harvesting Sources - (Question 5)</b>

In [25]:
# Selecting columns that represent the catch from source from the three DataFrames.
catch_source_df1 = fisher_df_1.iloc[:, 181:322]
catch_source_df2 = fisher_df_2.iloc[:, 251:392]
catch_source_df3 = fisher_df_3.iloc[:, 251:392]

# Concatenating the catch columns from all three DataFrames into one single DataFrame.
catch_source_df = pd.concat([
    catch_source_df1, catch_source_df2, catch_source_df3
])

# Resetting the index of the new DataFrame.
catch_source_df.reset_index(drop = True, inplace = True)

In [26]:
# Displaying the DataFrame's shape and size.
display(
    catch_source_df.shape,
    catch_source_df.head(5)
)

(7217, 141)

,q5,q5_1_n,q5_1_1,q5_1_2,q5_1_3,q5_1_4,q5_1_5,q5_1_6,q5_1_7,q5_1_8,...,q5_10_4,q5_10_5,q5_10_6,q5_10_7,q5_10_8,q5_10_9,q5_10_10,q5_10_11,q5_10_12,q5_10_t
0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Mapping the 'q5' column in the 'catch_source_df' DataFrame to the harvesting source.
catch_source_df['q5'] = catch_source_df['q5'].map(SOURCE)

# Filling any NaN values in the 'q5' column with 'Other'
catch_source_df['q5'].fillna('Others', inplace = True)

C:\Users\bapti\AppData\Local\Temp\ipykernel_31488\2690143327.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  catch_source_df['q5'].fillna('Others', inplace = True)


In [28]:
# Creating new columns for DataFrame.
new_columns = ['Source'] + MONTHS + ['Total']

# Creating a DataFrame with the above specified columns.
source_df = pd.DataFrame(columns = new_columns)

# Getting the unique values in the 'q5' column of the 'catch_source_df' DataFrame.
sources = catch_source_df['q5'].unique()

In [29]:
# Looping through each unique source in the 'catch_source_df' DataFrame.
for x in sources:
    # Skiping the iteration if the source is NaN.
    if pd.isna(x):

        continue

    data = [0] * 12  # List to hold monthly totals for the source.

    # Iterating through each month (1 -> 12) starting with April, ending with March.
    for month_idx in range(1, 13):
        month_total = 0

        # Iterating through the columns in the 'catch_source_df' DataFrame.
        for col in catch_source_df.columns:

            # Checking if the column name starts 'q5_', ends with the month index, and does not have 't'.
            if col.startswith(f'q5_') and col.endswith(f'_{month_idx}') and 't' not in col:
                # Summing the values in the column for the current source and add to the monthly total.
                month_total += catch_source_df[catch_source_df['q5'] == x][col].sum()

            # Updating the data list with the monthly total.
            data[month_idx - 1] = month_total

    year_total = sum(data)  # Calculating the total for the year.
    new_row = [x] + data + [year_total]  # Creating a new row with the source, monthly totals, and year total.
    source_df.loc[len(source_df)] = new_row  # Adding the new row to the 'source_df' DataFrame.

# Replace 'Unknown' with 'Other' in the 'Source' column.
source_df['Source'] = source_df['Source'].replace('Unknown', 'Others')

# Converting the monthly and yearly totals from kilograms to metric tonnes.
for col in source_df.columns[1:]:
    source_df[col] = (source_df[col] / 1000).round(2)

# Creating final DataFrame for exporting.
MONTHLY_TOTALS_BY_SOURCE_DF = source_df

In [30]:
# Displaying the DataFrame's shape and size.
display(
    MONTHLY_TOTALS_BY_SOURCE_DF.shape,
    MONTHLY_TOTALS_BY_SOURCE_DF.head(5)
)

(12, 14)

,Source,January--Magh,February--Falgun,March--Chaitra,April--Boishakh,May--Jeystho,June--Asharh,July--Srabon,August--Bhadro,September--Ashwin,October--Kartik,November--Aghrahan,December--Poush,Total
0,Others,191.99,138.58,150.21,131.57,225.58,246.34,250.09,286.85,211.53,211.80,188.28,121.67,2354.49
1,River (Cultivation),1168.42,830.65,851.66,992.79,1051.57,1478.46,3203.87,5441.44,5938.21,5455.31,3493.08,1194.67,31100.13
2,River,620.15,589.99,872.85,1101.94,1123.17,898.86,963.97,1011.01,962.48,961.51,664.19,449.65,10219.78
3,Marsh,64.16,20.87,34.63,74.72,107.34,199.26,272.14,211.11,165.84,249.57,57.62,31.70,1488.95
4,Seasonal Cultivation,1.09,0.98,1.50,27.48,25.08,17.77,63.04,55.80,124.64,62.84,18.44,1.02,399.68


# Q5_MONTHLY_TOTALS_BY_SOURCE.csv saved

In [31]:
# Saving the DataFrame to a CSV file.
MONTHLY_TOTALS_BY_SOURCE_DF.to_csv(
    'DATASETS/Cleaned_Data/Q5_MONTHLY_TOTALS_BY_SOURCE.csv',
    index = False
)

#### <b>Analysing Annual Wasteage Volumes and Species-Specific Wasteage Data - (Question 6)</b>

In [32]:
# Selecting columns that represent the waste from the three DataFrames.
annual_waste_totals1 = fisher_df_1.iloc[:, 322:434]
annual_waste_totals2 = fisher_df_2.iloc[:, 392:504]
annual_waste_totals3 = fisher_df_3.iloc[:, 392:504]

# Concatenating the waste columns from all three DataFrames into one single DataFrame.
annual_waste_totals = pd.concat([
    annual_waste_totals1, annual_waste_totals2, annual_waste_totals3
])

# Resetting the index of the new DataFrame.
annual_waste_totals.reset_index(drop = True, inplace = True)

In [33]:
# Displaying the DataFrame's shape and size.
display(
    annual_waste_totals.shape,
    annual_waste_totals.head(5)
)

(7217, 112)

,q6_1_n,q6_1_1,q6_1_2,q6_1_3,q6_1_4,q6_1_5,q6_1_6,q6_1_7,q6_1_8,q6_1_9,...,q6_8_4,q6_8_5,q6_8_6,q6_8_7,q6_8_8,q6_8_9,q6_8_10,q6_8_11,q6_8_12,q6_8_t
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0.0,0.0


In [34]:
# Looping through the range from (1 -> 10) to generate column names for q6_n
for x in range(1, 11):
    column_name = f'q6_{x}_n'

    # Checking if the column exists in the 'annual_waste_totals' DataFrame.
    if column_name in annual_waste_totals.columns:
        # Mapping the values in the column to fish labels using the FISH_LABELS dictionary.
        annual_waste_totals[column_name] = annual_waste_totals[column_name].map(FISH_LABELS)

# Looping through the columns in the 'annual_waste_totals' DataFrame.
for z in annual_waste_totals.columns:
    # Checking if the column name contains '_n'.
    if '_n' in z:
        # Filling NaN values in the column with 'Other Species'
        annual_waste_totals[z].fillna('Other Species', inplace = True)

C:\Users\bapti\AppData\Local\Temp\ipykernel_31488\238932882.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  annual_waste_totals[z].fillna('Other Species', inplace = True)


In [35]:
# Creating new columns for DataFrame.
new_columns = ['Fish Name'] + MONTHS + ['Year Total']

# Creating a DataFrame with the above specified columns.
annual_waste_df = pd.DataFrame(columns = new_columns)

In [36]:
# Looping through columns that contain '_n' in their names.
for x in annual_waste_totals.filter(like = '_n').columns:

    # Extracting the series number from the column name.
    series_number = x.split('_')[1]
    # Creating a list of column names for each month based on the series number.
    month_column = [f'q6_{series_number}_{z}' for z in range(1, 13)]

    # Iterating through the unique fish species names in the current column.
    for i in annual_waste_totals[x].unique():
        # Data list.
        data = []

        # Summing the waste data for each month and convert to metric tonnes.
        for col, month_name in zip(month_column, MONTHS):
            if col in annual_waste_totals.columns:
                month_info = (annual_waste_totals[annual_waste_totals[x] == i][col].sum() / 1000).round(2)
                data.append(month_info)
            else:
                data.append(0)

        # Calculating the total annual waste for the current species.
        year_total = sum(data)

        # Checking if the species already exists in the 'annual_waste_df' DataFrame.
        if i in annual_waste_df['Fish Name'].values:
            # If it exists, update the existing row.
            index = annual_waste_df[annual_waste_df['Fish Name'] == i].index[0]
            annual_waste_df.loc[index, MONTHS] += data
            annual_waste_df.loc[index, 'Year Total'] += year_total
        else:
            # If it does not exist, add a new row for the species.
            new_row = [i] + data + [year_total]
            annual_waste_df.loc[len(annual_waste_df)] = new_row

# Calculating the total waste for each month across all species.
month_totals_df = annual_waste_df.loc[:, 'January--Magh':'December--Poush'].sum()

# Sorting the DataFrame by the 'Year Total' column.
species_totals_df = annual_waste_df.sort_values(by = 'Year Total', ascending = False)

In [37]:
# Resetting the index of the 'month_totals_df'.
month_totals_df = month_totals_df.reset_index()

# Renaming the name of the columns.
month_totals_df.columns = ['Month', 'Total']

# Rounding the 'Total' column values to 2 decimal places.
month_totals_df['Total'] = month_totals_df['Total'].round(2)

# Creating final DataFrame for exporting.
MONTHLY_WASTE_DF = month_totals_df


In [38]:
# Rounding all numerical columns in the 'species_totals_df' DataFrame to 2 decimal places.
species_totals_df = species_totals_df.iloc[:, 0:].round(2)

# Sorting the DataFrame by 'Year Total' column in descending order.
species_totals_df = species_totals_df.sort_values(by = 'Year Total', ascending = False)

# Removing rows where the 'Fish Name' column is 'Other Species'.
species_totals_df = species_totals_df[species_totals_df['Fish Name'] != 'Other Species']

# Selecting the Top 10 species based on the 'Year Total'.
MONTHLY_FISH_WASTE_DF = species_totals_df.head(10)

In [39]:
# Displaying the DataFrame's shape and size.
display(
    MONTHLY_WASTE_DF.shape,
    MONTHLY_WASTE_DF.head(5)
)

(12, 2)

,Month,Total
0,January--Magh,18.04
1,February--Falgun,16.56
2,March--Chaitra,13.89
3,April--Boishakh,17.37
4,May--Jeystho,19.61


In [40]:
# Displaying the DataFrame's shape and size.
display(
    MONTHLY_FISH_WASTE_DF.shape,
    MONTHLY_FISH_WASTE_DF
)

(10, 14)

,Fish Name,January--Magh,February--Falgun,March--Chaitra,April--Boishakh,May--Jeystho,June--Asharh,July--Srabon,August--Bhadro,September--Ashwin,October--Kartik,November--Aghrahan,December--Poush,Year Total
9,Tilapia,2.01,2.00,2.19,3.15,5.11,7.35,8.26,2.90,1.09,1.27,2.43,9.64,47.40
4,Silver carp,1.58,0.96,0.67,0.59,0.96,1.25,2.12,1.76,0.70,0.57,1.17,3.42,15.75
1,Mrigel,1.42,1.35,0.48,1.14,0.53,0.75,0.92,1.09,0.48,0.48,1.00,2.65,12.29
2,Rui,0.60,0.67,0.15,0.89,0.29,0.38,0.49,0.82,0.46,0.52,0.60,1.88,7.75
3,Taki,0.24,0.76,0.96,0.82,0.91,0.95,0.73,0.30,0.08,0.07,0.19,0.22,6.23
16,Bata,0.70,0.34,0.25,0.42,0.20,0.28,0.21,0.35,0.16,0.50,0.48,1.92,5.81
8,Pangas,0.93,0.28,0.25,0.09,0.16,0.21,1.38,0.42,0.26,0.54,0.38,0.51,5.41
21,Hilsha,0.64,0.27,0.38,1.14,1.10,0.69,0.52,0.30,0.04,0.03,0.12,0.16,5.39
13,Other small shrimp/prawn,0.74,0.58,0.51,0.52,0.48,0.47,0.35,0.16,0.07,0.09,0.63,0.76,5.36
10,Singh,0.50,0.44,0.27,0.21,0.20,1.06,0.32,0.22,0.14,0.14,0.24,0.98,4.72


# Q6_MONTHLY_WASTE.csv saved

In [41]:
# Saving the DataFrame to a CSV file.
MONTHLY_WASTE_DF.to_csv(
    'DATASETS/Cleaned_Data/Q6_MONTHLY_WASTE.csv',
    index = False
)

In [42]:
# Saving the DataFrame to a CSV file.
MONTHLY_FISH_WASTE_DF.to_csv(
    'DATASETS/Cleaned_Data/Q6_MONTHLY_FISH_WASTE.csv',
    index = False
)

#### <b>Examining Specific Causes of Fish Waste - (Question 7)</b>

In [43]:
# Selecting columns that represent the cause of waste from the three DataFrames.
waste_reason_df1 = fisher_df_1.iloc[:, 434:714]
waste_reason_df2 = fisher_df_2.iloc[:, 504:784]
waste_reason_df3 = fisher_df_3.iloc[:, 504:784]

# Concatenating the cause of waste columns from all three DataFrames into one single DataFrame.
waste_reason_df = pd.concat([
    waste_reason_df1, waste_reason_df2, waste_reason_df3
])

# Resetting the index of the new DataFrame.
waste_reason_df.reset_index(drop = True, inplace = True)

In [44]:
# Displaying the DataFrame's shape and size.
display(
    waste_reason_df.shape,
    waste_reason_df.head(5)
)

(7217, 280)

,q7_1_n,q7_1_o_1,q7_1_o_2_1,q7_1_o_2_2,q7_1_o_3_1,q7_1_o_3_2,q7_1_ob_1,q7_1_ob_21,q7_1_ob_22,q7_1_ob_31,...,q7_10_b_1,q7_10b_2_1,q7_10b_2_2,q7_10b_3_1,q7_10b_3_2,q7_10_b_4,q7_10_l_i1,q7_10_l_i2,q7_10_l_c1,q7_10_l_c2
0,1,0.0,0,0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0,0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0,0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0,0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0,0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# Looping through the range from (1 -> 10) to generate column names for 'q7_n'.
for x in range(1, 11):
    column_name = f'q7_{x}_n'

    # Checking if the column exists in the 'waste_reason_df' DataFrame.
    if column_name in waste_reason_df.columns:
        # Mapping the values in the column to fish labels using the FISH_LABELS dictionary.
        waste_reason_df[column_name] = waste_reason_df[column_name].map(FISH_LABELS)

# Looping through the columns in the 'waste_reason_df' DataFrame.
for z in waste_reason_df.columns:
    # Checking if the column name contains '_n'.
    if '_n' in z:
        # Filling NaN values in the column with 'Other Species'.
        waste_reason_df[z].fillna('Other Species', inplace = True)

C:\Users\bapti\AppData\Local\Temp\ipykernel_31488\3659323892.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  waste_reason_df[z].fillna('Other Species', inplace = True)


In [46]:
data = []

# Looping through the range from (1 -> 10) to process the columns related to fish waste reasons.
for x in range(1, 11):

    # Generating the column names based on the current iteration.
    fish_name = f'q7_{x}_n'
    quantity_1 = f'q7_{x}_o_1'
    reason_1 = f'q7_{x}_o_2_1'
    reason_2 = f'q7_{x}_o_2_2'
    quantity_2 = f'q7_{x}_o_3_1'

    # Listing the required columns.
    required_cols = [fish_name, quantity_1, reason_1, reason_2, quantity_2]

    # Checking if all required columns exist in the 'waste_reason_df' DataFrame.
    if all(col in waste_reason_df.columns for col in required_cols):
        data_check = waste_reason_df[required_cols]
        # Renaming the columns.
        data_check.columns = ['Fish Name', 'Quantity_Lost_mt', 'Reason_1', 'Reason_2', 'Quantity_Lost_Reasons_mt']

        data.append(data_check)

# Concatenating all the data from the list into a single DataFrame.
new_data = pd.concat(data)

# Mapping the 'Reason_1' and 'Reason_2' columns to their descriptions using the REASONS dictionary.
new_data['Reason_1'] = new_data['Reason_1'].map(REASONS)
new_data['Reason_2'] = new_data['Reason_2'].map(REASONS)

# Filtering the DataFrame to keep only rows where either quantity lost column is greater than 0.
new_data = new_data[
    (new_data['Quantity_Lost_mt'] > 0) |
    (new_data['Quantity_Lost_Reasons_mt'] > 0)
]

# Reshaping the DataFrame by melting it, transforming multiple reason columns into rows.
new_data = pd.melt(
    new_data,
    id_vars = ['Fish Name', 'Quantity_Lost_mt', 'Quantity_Lost_Reasons_mt'],
    value_vars = ['Reason_1', 'Reason_2'],
    var_name = 'Reason_type',
    value_name = 'Reason'
)

# Droping rows where the 'Reason' column is NaN.
new_data = new_data.dropna(subset = ['Reason'])

# Grouping by the 'Reason' column and aggregate the total quantity lost for each reason.
final_df = new_data.groupby('Reason').agg(
    total_quantity_lost_mt = ('Quantity_Lost_Reasons_mt', 'sum')
).reset_index()

# Converting the total quantity lost from kilograms to metric tonnes.
final_df['total_quantity_lost_mt'] = final_df['total_quantity_lost_mt'] / 1000

# Sort the DataFrame by the total quantity lost in descending order
ANNUAL_LOSS_BY_REASON_DF = final_df.sort_values(by = 'total_quantity_lost_mt', ascending = False).round(2)

In [47]:
# Displaying the DataFrame's shape and size.
display(
    ANNUAL_LOSS_BY_REASON_DF.shape,
    ANNUAL_LOSS_BY_REASON_DF.head(5)
)

(11, 2)

,Reason,total_quantity_lost_mt
1,"High Temperature, Delay In Taking To Market",65.53
0,Damage During Harvesting,29.88
10,Too Long In Nets (Physical Damage),28.80
7,Other Unrecorded Reason For Loss,19.61
9,Spoilage of Fish During Transportation,16.44


Q7_ANNUAL_LOSS_BY_REASON.csv saved

In [48]:
# Saving the DataFrame to a CSV file.
ANNUAL_LOSS_BY_REASON_DF.to_csv(
    'DATASETS/Cleaned_Data/Q7_ANNUAL_LOSS_BY_REASON.csv',
    index = False
)

#### <b>Distribution Channels of the Fish - (Question 12)</b>

In [49]:
# Selecting columns that represent the distribution channels of fish from the three DataFrames.
fish_sold_df1 = fisher_df_1.iloc[:, 792:1114]
fish_sold_df2 = fisher_df_2.iloc[:, 891:1213]
fish_sold_df3 = fisher_df_3.iloc[:, 891:1213]

# Concatenating the cause of waste columns from all three DataFrames into one single DataFrame.
fish_sold_df = pd.concat([
    fish_sold_df1, fish_sold_df2, fish_sold_df3
])

# Resetting the index of the new DataFrame.
fish_sold_df.reset_index(drop = True, inplace = True)

In [50]:
# Displaying the DataFrame's shape and size.
display(
    fish_sold_df.shape,
    fish_sold_df.head(5)
)

(7217, 327)

,q12_b1_nam,q12_b1_f_k,q12_b1_f_t,q12_b1_b_k,q12_b1_b_t,q12_b1_p_k,q12_b1_p_t,q12_b1_a_k,q12_b1_a_t,q12_b1_c_k,...,q12_b14_d_t,q12_b14_ac_k,q12_b14_ac_t,q12_b14_r_k,q12_b14_r_t,q12_b10_a_k,q12_b10_a_t,q12_b10_c_k,q12_b10_c_t,q12_b10_k_k
0,1,0.0,0,0.0,0,0.0,0,0,0,20400.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0,0.0,0,0.0,0,0,0,8000.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,1,0.0,0,0.0,0,0.0,0,0,0,10500.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,1,0.0,0,0.0,0,0.0,0,0,0,14000.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,1,0.0,0,0.0,0,0.0,0,0,0,5500.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [51]:
# Looping through the range from (1 -> 15) to generate column names for 'q12_bx_nam'.
for x in range(1, 16):
    column_name = f'q12_b{x}_nam'

    # Checking if the column exists in the 'fish_sold_df' DataFrame.
    if column_name in fish_sold_df.columns:
        # Mapping the values in the column to fish labels using the FISH_LABELS dictionary.
        fish_sold_df[column_name] = fish_sold_df[column_name].map(FISH_LABELS)

# Looping through the columns in the 'fish_sold_df' DataFrame.
for z in fish_sold_df.columns:
    # Checking if the column name contains '_n'.
    if '_n' in z:
        # Filling NaN values in the column with 'Other Species'.
        fish_sold_df[z].fillna('Other Species', inplace = True)

C:\Users\bapti\AppData\Local\Temp\ipykernel_31488\2348583712.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fish_sold_df[z].fillna('Other Species', inplace = True)


In [52]:
# Identifying columns that end with '_t' and store them in a list.
columns_to_drop = [col for col in fish_sold_df.columns if col.endswith('_t')]

# Droping the identified columns from the 'fish_sold_df' DataFrame.
data_cleaned = fish_sold_df.drop(columns=columns_to_drop)

# Identifying columns that end with '_k' and store them in a list.
columns_to_sum = [col for col in data_cleaned.columns if col.endswith('_k')]

# Grouping the data by the 'q12_b1_nam' column and summing the identified columns.
summary = data_cleaned.groupby('q12_b1_nam')[columns_to_sum].sum()

totals_by_letter = {}

# Looping through each column in the 'columns_to_sum' list.
for column in columns_to_sum:

    # Extracting the letter part from the column name.
    letter_part = column.split('_')[-2]

    # Summing the values of the column and updating the 'totals_by_letter' dictionary.
    if letter_part not in totals_by_letter:
        totals_by_letter[letter_part] = data_cleaned[column].sum()
    else:
        totals_by_letter[letter_part] += data_cleaned[column].sum()

totals_by_letter

{'f': np.float64(1476427.1),
 'b': np.float64(11142899.45),
 'p': np.float64(3685383.46),
 'a': np.float64(1998972.7),
 'c': np.float64(58237011.559999995),
 'k': np.float64(5094295.749999999),
 'v': np.float64(1434549.29),
 'h': np.int64(1514),
 'd': np.int64(231),
 'ac': np.int64(18),
 'r': np.int64(14094)}

In [53]:
breakdown_by_letter_and_fish = {}

# Looping through each column in the 'columns_to_sum' list.
for column in columns_to_sum:

    # Extracting the letter part from the column name.
    letter_part = column.split('_')[-2]

    # Grouping the data by 'q12_b1_nam' and summing the values in the current column.
    group_sum = data_cleaned.groupby('q12_b1_nam')[column].sum()

    # If the letter part is not already in the dictionary, add it with the group sum as a DataFrame.
    if letter_part not in breakdown_by_letter_and_fish:
        breakdown_by_letter_and_fish[letter_part] = group_sum.to_frame(name = column)
    else:
        # If the letter part is already in the dictionary, join the new group sum DataFrame to the existing one.
        breakdown_by_letter_and_fish[letter_part] = breakdown_by_letter_and_fish[letter_part].join(group_sum)

# Create a dictionary by summing the columns for each key in the 'breakdown_by_letter_and_fish' dictionary.
final_breakdown = {key: df.sum(axis = 1) for key, df in breakdown_by_letter_and_fish.items()}

# Converting the final breakdown dictionary into a DataFrame.
final_breakdown_df = pd.DataFrame(final_breakdown)

In [54]:
# Renaming the columns of 'final_breakdown_df' using the DISTRIBUTION dictionary.
final_breakdown_renamed = final_breakdown_df.rename(columns = DISTRIBUTION)

# Adding a new column 'Total (mt)' by summing the values across all columns for each row.
final_breakdown_renamed['Total (mt)'] = final_breakdown_renamed.sum(axis = 1)

# Sorting the DataFrame by the 'Total (mt)' column.
final_breakdown_renamed = final_breakdown_renamed.sort_values(by = 'Total (mt)', ascending = False)

# Converting the values from kilograms to metric tonnes.
final_breakdown_renamed = (final_breakdown_renamed / 1000)

# Selecting the top 10 rows based on 'Total (mt)' and round the values to 2 decimal places
WHERE_DOES_THE_FISH_END_UP_DF = final_breakdown_renamed.head(10).round(2)

In [55]:
# Displaying the DataFrame's shape and size.
display(
    WHERE_DOES_THE_FISH_END_UP_DF.shape,
    WHERE_DOES_THE_FISH_END_UP_DF
)

(10, 12)

,Local Agent,Wholesaler (Dealer),Wholesaler (Wholesale),Wholesaler (Merchant),Merchant (Commission Agent),Retailer,Consumer,Hotel Restaurant,Depot Owner,Account Holder,Exporter,Total (mt)
q12_b1_nam,,,,,,,,,,,,
Rui,116.06,3275.78,1347.40,394.36,20822.87,1433.52,169.07,0.20,0.00,0.00,0.08,27559.33
Tilapia,189.54,2176.16,408.64,232.24,12912.39,1857.95,63.88,0.00,0.00,0.00,0.00,17840.79
Pangas,563.66,2213.77,669.51,274.39,4129.47,651.96,18.83,0.00,0.00,0.00,0.00,8521.59
Hilsha,31.30,613.57,252.78,749.35,6357.82,38.89,145.86,0.00,0.00,0.00,0.01,8189.58
Other Species,24.66,428.83,314.84,121.29,3063.81,312.78,447.54,0.38,0.00,0.00,0.00,4714.14
Pabda,2.70,507.00,188.47,58.28,861.86,116.64,3.74,0.00,0.00,0.00,0.00,1738.69
Silver carp,17.35,183.68,29.15,7.42,1263.22,142.74,12.59,0.00,0.00,0.00,0.00,1656.15
Koi,32.80,653.69,129.03,4.95,758.11,10.22,22.90,0.00,0.00,0.00,0.00,1611.71
Singh,6.52,457.54,81.79,43.42,716.36,21.37,17.25,0.00,0.00,0.00,14.00,1358.23


In [56]:
# Saving the DataFrame to a CSV file.
WHERE_DOES_THE_FISH_END_UP_DF.to_csv(
    'DATASETS/Cleaned_Data/Q12_WHERE_DOES_THE_FISH_END_UP.csv'
)

In [57]:
print("=== final_df Schema ===")
schema_df = pd.DataFrame({
    'column': final_df.columns,
    'dtype': [final_df[col].dtype for col in final_df.columns]
})
display(schema_df)

=== final_df Schema ===


,column,dtype
0,Reason,object
1,total_quantity_lost_mt,float64


In [58]:
print("=== Sample of final_df ===")
display(final_df)

=== Sample of final_df ===


,Reason,total_quantity_lost_mt
0,Damage During Harvesting,29.88230
1,"High Temperature, Delay In Taking To Market",65.53131
2,Inadequacy of Communication Systems,1.99900
3,Inadequacy of Fish Preservation Materials,3.66630
4,Inadequate Cold Storage Facilities,1.03200
5,Loss of Fish From Unloading & Loading,4.84500
6,Not Enough Ice or Insulated Containers,4.68050
7,Other Unrecorded Reason For Loss,19.60620
8,Result of Medication Used On Fish,14.54500
9,Spoilage of Fish During Transportation,16.44450


In [59]:
print("All data has been exported to 'DATASETS/Cleaned_Data/...")

All data has been exported to 'DATASETS/Cleaned_Data/...
